# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,preobrazheniye,42.9019,133.9064,18.97,53,100,2.19,RU,2024-09-16 02:50:08
1,1,bolpur,23.6667,87.7167,25.77,91,100,8.60,IN,2024-09-16 02:50:09
2,2,eirunepe,-6.6603,-69.8736,29.70,44,75,0.97,BR,2024-09-16 02:50:10
3,3,college,64.8569,-147.8028,10.99,73,75,2.57,US,2024-09-16 02:47:03
4,4,amungan,15.3652,119.9582,27.85,82,100,4.33,PH,2024-09-16 02:50:12
5,5,aykhal,66.0000,111.5000,0.64,81,100,3.37,RU,2024-09-16 02:50:13
6,6,vorkuta,67.5000,64.0000,13.07,97,99,4.61,RU,2024-09-16 02:50:14
7,7,port-aux-francais,-49.3500,70.2167,2.83,82,99,15.16,TF,2024-09-16 02:50:15
8,8,i-n-salah,27.1935,2.4607,32.99,18,0,3.99,DZ,2024-09-16 02:50:16
9,9,atafu village,-8.5421,-172.5159,28.23,76,54,3.36,TK,2024-09-16 02:50:18


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.7

)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= 80) & 
    (city_data_df["Max Temp"] >= 65) & 
    (city_data_df["Humidity"] <= 40) &
    (city_data_df["Cloudiness"] <= 25) &
    (city_data_df["Wind Speed"] <= 5)
]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df.copy(deep=True)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories": categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index,"Lng"]
    latitude = hotel_df.loc[index,"Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
        
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 100,
    color = "City",
    alpha = 1,
    hover_cols = ["Hotel Name","Country"]

)

# Display the map
hotel_map


ValueError: cannot convert float NaN to integer

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)